In [140]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import json
import konlpy
from nltk.tokenize import word_tokenize
import re
from konlpy.tag import Okt
import time
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from itertools import combinations
from itertools import product

text=[["a", "b", "c", "d"], ["가", "다", "a", "b"], ["a", "d", "b", "c"], ["c", "d", "b", "e", "a"], ["c", "b", "a"], ["d", "c"]]

train, vocab=padded_everygram_pipeline(2, text)
vocab=list(vocab)
ngram=MLE(3)
ngram.fit(train,vocab)

In [3]:
type(ngram.counts[["a"]]["b"])

int

In [4]:
#model, 이전 단어, 이후 단어를 바탕으로 bigram probs를 연산하는 모델 생성
def bigram_probs(model, pre_word, post_word):
    word_dict=dict(model.counts[[pre_word]])
    wholecount=model.counts[pre_word]
    try:
        post_count=word_dict[post_word]
    except:
        post_count=0
    emer_prob=post_count/wholecount
    return emer_prob


In [5]:
emer_prb=bigram_probs(ngram, "a", "가")

In [6]:
emer_prb

0.0

In [7]:
#transitive closure 활용해서 trigram probs 연산하는 모델 생성
#trigram probs 연산이 bigram보다 단연 안좋아질 수도 있으니 이에 대한 normalization 작업 수행
def trigram_probs(model, pre_word, post_word):
    tulist=[]
    word_dict=dict(model.counts[[pre_word]])
    
    for key in list(word_dict.keys()):
        if key !=pre_word:
            first_prob=bigram_probs(model, pre_word, key)
            second_prob=bigram_probs(model, key, post_word)
            whole_prob=first_prob*second_prob
            if whole_prob!=0:
                #normalization
                whole_prob=whole_prob*2
                tulist.append([pre_word, key, post_word, whole_prob])
        else:
            continue
    return tulist

In [8]:
worddict=trigram_probs(ngram, "a", "c")

In [9]:
worddict

[['a', 'b', 'c', 0.32000000000000006]]

In [10]:
#pre_word 와 post_word를 바탕으로 bigram 해보고 안되면 trigram까지 하는 함수 생성
def canconnect(model, pre_word, post_word):
    possiblelist=[]
    biprob=bigram_probs(model, pre_word, post_word)
    if biprob!=0:
        possiblelist.append([pre_word, post_word, biprob])
    
    tulist=trigram_probs(model, pre_word, post_word)
    for li in tulist:
        possiblelist.append(li)
    return possiblelist

In [11]:
canconnect(ngram, "a", "b")
#시작 단어, 가운데 단어(있다면)**, 끝 단어, 등장확률

[['a', 'b', 0.4], ['a', 'd', 'b', 0.26666666666666666]]

In [14]:
import itertools
given_list=["a", "b", "c", "d"]
#list(itertools.permutations(given_list, 4))

In [23]:
def bigconnect(model, list_of_words):
    dummylist=[]
    index_list=[]
    for i in range(len(list_of_words)-1):
        connection=canconnect(model, list_of_words[i], list_of_words[i+1])
        if len(connection)!=0:
            n=len(connection)
            index_list.append(n)
            for possible in connection:
                dummylist.append(possible)
    
        else:
            print("connection failed")
            break
    return dummylist, index_list
        
    

In [24]:
canconnect(ngram, "a", "b")[0][:-1]

['a', 'b']

In [37]:
bigconnect(ngram, ["a", "가"])

connection failed


[]

In [25]:
bigconnect(ngram, ["c", "b", "a", "d", "e"])

([['c', 'b', 0.25],
  ['c', 'd', 'b', 0.6666666666666666],
  ['b', 'a', 0.2],
  ['b', 'e', 'a', 0.4],
  ['a', 'd', 0.2],
  ['d', 'b', 'e', 0.26666666666666666]],
 [2, 2, 1, 1])

In [34]:
doublelist, index_list=bigconnect(ngram, ["c", "b", "a", "d", "e"])


In [74]:
def process(model, list_of_words):
    doublelists, index=bigconnect(model, list_of_words)
    for indexes in range(len(doublelists)):
        doublelists[indexes]=tuple(doublelists[indexes])
    return doublelists, index
    



In [75]:
lists, indexes=process(ngram, ["c", "b", "a", "d", "e"])

In [76]:
lists

[('c', 'b', 0.25),
 ('c', 'd', 'b', 0.6666666666666666),
 ('b', 'a', 0.2),
 ('b', 'e', 'a', 0.4),
 ('a', 'd', 0.2),
 ('d', 'b', 'e', 0.26666666666666666)]

In [77]:
indexes

[2, 2, 1, 1]

In [78]:
def separate_connection(model,list_of_words):
    newconlist=[]
    doublelists, index=process(model, list_of_words)
    end, start=0, 0
    for i in index:
        end+=i
        newconlist.append(doublelists[start:end])

        start+=i
    return newconlist
    
                

In [79]:
newlist=separate_connection(ngram, ["c", "b", "a", "d", "e"])

In [131]:
def make_it_one(model, list_of_words):
    wholeconcat=[]
    wholeprob=1.0
    newlist=separate_connection(ngram, ["c", "b", "a", "d", "e"])
    maybe_one=list(product(*newlist))
    for i in maybe_one:
        for index in range(len(i)):
            wholeconcat.append(i[index][:-2])
            wholeprob*=i[index][-1]
        wholeconcat.append(list_of_words[-1])
        
        wholeconcat.append(wholeprob)
        wholeprob=1.0
    return wholeconcat
        
    

In [138]:
make_it_one(ngram, ["c", "b", "a", "d", "e"])

[('c',),
 ('b',),
 ('a',),
 ('d', 'b'),
 'e',
 0.002666666666666667,
 ('c',),
 ('b', 'e'),
 ('a',),
 ('d', 'b'),
 'e',
 0.005333333333333334,
 ('c', 'd'),
 ('b',),
 ('a',),
 ('d', 'b'),
 'e',
 0.0071111111111111115,
 ('c', 'd'),
 ('b', 'e'),
 ('a',),
 ('d', 'b'),
 'e',
 0.014222222222222223]

In [141]:
make_it_one(ngram, ["c", "b", "a", "d", "e"])

[('c',),
 ('b',),
 ('a',),
 ('d', 'b'),
 'e',
 0.0016000000000000005,
 ('c',),
 ('b', 'e'),
 ('a',),
 ('d', 'b'),
 'e',
 0.003200000000000001,
 ('c', 'd'),
 ('b',),
 ('a',),
 ('d', 'b'),
 'e',
 0.003200000000000001,
 ('c', 'd'),
 ('b', 'e'),
 ('a',),
 ('d', 'b'),
 'e',
 0.006400000000000002]